<a href="https://colab.research.google.com/github/luciabarg/datos_tumoresferas/blob/main/datos_tumoresferas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h4> Datos de tumoresferas </h4>

En una colaboración con el Laboratorio de Células Madre del IBYME nos enviaron imágenes tomadas usando un microscopio óptico invertido
común, en el cuarto de cultivos del Laboratorio de Células Madre del IBYME
por la Lic. en Ciencias Biológicas Antonella Vila, becaria del IBYME-CONICET
bajo la dirección del Dr. Vellón.


Para extraer la información estructurada a partir de las imágenes hicimos uso del programa Fiji, que es un paquete de procesamiento de imágenes, distribución muy completa del programa [ImageJ](https://www.medicinatv.com/profesional/reportajes/una-plataforma-abierta-revoluciona-el-procesamiento-de-imagenes-biomedicas-4596/), que es ampliamente utilizado para procesar
imágenes médicas. De acuerdo a la documentación del programa,
es posible contar y medir objetos si conocemos la escala en la cual se han
llevado a cabo dichas imágenes, en nuestro caso, a cuántos $\mu$m equivale un
píxel.
Para llevar a cabo el procedimiento detallado en la [documentación](https://imagej.nih.gov/ij/docs/guide/146-30.html#sub:Analyze-Particles...),
es necesario contar con una imagen umbralizada.



In [ ]:
from IPython.display import Image
Image(filename="/content/drive/MyDrive/Datos_tumoresferas/ejempo_segmentacion.png")

Importamos las librerías necesarias:

In [ ]:
import os
import numpy as np
import pandas as pd
from random import random
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import fmin
from scipy.integrate import solve_ivp
import seaborn as sns
import matplotlib.ticker as ticker
import warnings
warnings.filterwarnings("ignore")

from matplotlib import rc, rcParams
rcParams['patch.force_edgecolor'] = True
rcParams['patch.facecolor'] = 'b'
pd.options.display.float_format = '{:,.2f}'.format

Para asociar el Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/My Drive/Datos_tumoresferas")

# para acceder a la carpeta, en el siguiente link:
# https://drive.google.com/file/d/1WgsbqeSG2kOoHZDyLhk2IZ5FgB1Festk/view?usp=sharing

## Datos


Cargamos los datos de los experimentos realizados en el IBYME desde el día cero al séptimo. 


Ya se encuentran las etiquetas y el valor de las poblaciones estimadas, tomando como D1 = 18.34 µm al valor mínimo del diámetro de las células segmentadas.

In [ ]:
fiji_datos = pd.read_csv("fiji_datos_0al7mo_labels.csv") 
print('cantidad de datos:', len(fiji_datos))
fiji_datos.head()

cantidad de datos: 1018


,labels,Area,X,Y,XM,YM,Perim.,BX,BY,Width,...,FeretY,FeretAngle,MinFeret,AR,Round,Solidity,Esferoide,dia,Diameter,n_diam
0,Esferas_BT474_dia_0_well_1_100X_1_blob_1,324.44,"1,129.27",102.21,"1,129.27",102.21,67.36,"1,118.89",92.39,21.06,...,157,23.43,19.70,1.07,0.94,0.95,si,0,20.96,1.49
1,Esferas_BT474_dia_0_well_1_100X_1_blob_2,497.51,"1,517.55",126.10,"1,517.55",126.10,82.80,"1,504.76",113.45,25.82,...,192,16.14,25.13,1.05,0.95,0.96,si,0,26.00,2.85
2,Esferas_BT474_dia_0_well_1_100X_1_blob_3,282.91,"1,314.28",126.10,"1,314.28",126.10,62.16,"1,304.35",116.85,19.70,...,189,15.42,18.34,1.04,0.96,0.95,si,0,19.39,1.18
3,Esferas_BT474_dia_0_well_1_100X_1_blob_5,500.74,"1,189.28",212.62,"1,189.28",212.62,84.56,"1,175.95",200.41,26.49,...,309,174.14,25.14,1.03,0.97,0.95,si,0,25.88,2.81
4,Esferas_BT474_dia_0_well_1_100X_1_blob_6,492.90,"1,043.08",247.77,"1,043.08",247.77,82.07,"1,030.57",235.73,25.14,...,353,143.43,24.46,1.02,0.98,0.96,si,0,25.34,2.64
